In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, TimeDistributed, concatenate
from keras.models import Model,Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import TensorBoard
from keras import metrics

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from process_data import load_data
from data_util import DATE_INTERVAL_NEWS,MAX_NEWS_NUM,EMBEDDING_SIZE

In [ ]:
data = load_data(['GOOGL'])

In [ ]:
data['GOOGL']['train'][0].shape

# baseline model

In [ ]:
def build_baseline_model(code='Default'):
    news_input = Input(shape=(DATE_INTERVAL_NEWS,MAX_NEWS_NUM,EMBEDDING_SIZE))
    day_layer = GRU(50, return_sequences=False)
    day_layer = TimeDistributed(day_layer)
    inv_layer = GRU(50, return_sequences=False)
    x = day_layer(news_input)
    x = inv_layer(x)
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs=news_input,outputs=x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
model = build_baseline_model()

In [ ]:
print(model.summary())

In [ ]:
model.fit(x=data['GOOGL']['train'][0],y=data['GOOGL']['train'][2],batch_size=16,epochs=200,verbose=1,\
          validation_data=(data['GOOGL']['val'][0],data['GOOGL']['val'][2]),\
          callbacks=[TensorBoard(log_dir='model_log/')])

In [ ]:
model.evaluate(x=x_test, y=y_test,batch_size=16)

# NUMERICAL MODEL

In [ ]:
numerical_timestep = 20 #  correspond to the 'size' of  the window
attribute_num = 5 # Open/High/Low/AdjClose/Volume 

def build_numerical_model(code='Default'):
    numerical_input = Input(shape=(numerical_timestep,attribute_num))
    x = GRU(50,return_sequences=True)(numerical_input)
    x = Dropout(0.2)(x)
    x = GRU(50)(x)
    x = Dropout(0.2)(x)
    # x = Dense(10)(x)
    x = Dense(2,activation='softmax')(x)
    model = Model(inputs=numerical_input,outputs=x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
nmodel = build_numerical_model()

In [ ]:
print(model.summary())

In [ ]:
nmodel.fit(x=data['GOOGL']['train'][1],y=data['GOOGL']['train'][2],batch_size=16,epochs=50,verbose=1,\
          validation_data=(data['GOOGL']['val'][1],data['GOOGL']['val'][2]),\
          callbacks=[TensorBoard(log_dir='num_log/')])